In [ ]:
# Load libraries
import ruamel.yaml as ry
import matplotlib.pyplot as plt
import numpy as np
import os

def load_yaml(fname_input):
    reader = ry.YAML(typ="safe", pure=True)
    with open(fname_input, "r", encoding="utf-8") as f:
        input_yaml = reader.load(f)
    return input_yaml

In [ ]:
# Load windIO output yaml files 
paths = {"HAWC2":   {"path": "HAWC2/iea-22-280-rwt-steady-states-hawc2.yaml"},
         "OpenFAST": {"path": "OpenFAST/iea-22-280-rwt-steady-states-of.yaml"}}
data = {}


for name, config in paths.items():
    data[name] = load_yaml(config["path"])

In [ ]:
def get_floats_from_dict(cases, subdict, name):
    return [case[subdict][name] for case in cases]

In [ ]:
# Create dictionaries of numpy arrays for easy plots
wind_speed = {}
rotor_speed = {}
pitch = {}

for i, (name, dataset) in enumerate(data.items()):
    cases = dataset['cases']
    n_ws = len(cases)
    wind_speed[name] = [case["configuration"]["wind_speed"] for case in cases]
    pitch[name] = [case["configuration"]["pitch"] for case in cases]
    rotor_speed[name] = [case["configuration"]["rotor_speed"] for case in cases]


In [ ]:
font_size = 10
f, ax = plt.subplots(1, 1, figsize=(5.3, 4))
for i, (name, dataset) in enumerate(data.items()):
    ax.plot(wind_speed[name], rotor_speed[name], label=name)
ax.set_xlabel("Wind Speed (m/s)", fontsize=font_size + 2, fontweight="bold")
ax.set_ylabel("Rotor Speed (rpm)", fontsize=font_size + 2, fontweight="bold")
ax.grid(color=[0.8, 0.8, 0.8], linestyle="--")
ax.legend()
plt.subplots_adjust(bottom=0.15, left=0.15)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(5.3, 4))
for i, (name, dataset) in enumerate(data.items()):
    ax.plot(wind_speed[name], pitch[name], label=name)
ax.set_xlabel("Wind Speed (m/s)", fontsize=font_size + 2, fontweight="bold")
ax.set_ylabel("Pitch (deg)", fontsize=font_size + 2, fontweight="bold")
ax.grid(color=[0.8, 0.8, 0.8], linestyle="--")
ax.legend()
plt.subplots_adjust(bottom=0.15, left=0.15)

In [ ]:
integrated_keys = list(data["HAWC2"]["cases"][0]["outputs"]["integrated"].keys())
distributed_keys = list(data["HAWC2"]["cases"][0]["outputs"]["distributed"].keys())

for key in integrated_keys:
    f, ax = plt.subplots(1, 1, figsize=(5.3, 4))
    for i, (name, dataset) in enumerate(data.items()):
        values = [case["outputs"]["integrated"][key] for case in dataset["cases"]]
        ax.plot(wind_speed[name], values, label=name)
    ax.set_xlabel("Wind Speed (m/s)", fontsize=font_size + 2, fontweight="bold")
    ax.set_ylabel("%s" % key, fontsize=font_size + 2, fontweight="bold")
    ax.grid(color=[0.8, 0.8, 0.8], linestyle="--")
    ax.legend()
    plt.subplots_adjust(bottom=0.15, left=0.15)

In [ ]:
len(data['HAWC2']['cases'][7]["outputs"]["distributed"]["blade_Fz_pitching"]["values"])

In [ ]:
distributed_keys = list(data["HAWC2"]["cases"][0]["outputs"]["distributed"].keys())
ws = 10.0
for key in distributed_keys:
    f, ax = plt.subplots(1, 1, figsize=(5.3, 4))
    ax.set_title("Wind speed = %2.2f" % ws)
    for i, (name, dataset) in enumerate(data.items()):
        try:
            case_ix = wind_speed[name].index(ws)
            case = dataset["cases"][case_ix]
            grid = case["outputs"]["distributed"][key]["grid"]
            values = case["outputs"]["distributed"][key]["values"]
            ax.plot(grid, values, label=name)
        except:
            print("Wind speed %2.2f not in dataset: %s for %s" % (ws, name, key)) 
    ax.set_xlabel("Spanwise grid (-)", fontsize=font_size + 2, fontweight="bold")
    ax.set_ylabel("%s" % key, fontsize=font_size + 2, fontweight="bold")
    ax.grid(color=[0.8, 0.8, 0.8], linestyle="--")
    ax.legend()
    plt.subplots_adjust(bottom=0.15, left=0.15)
    plt.savefig('%s.png' % key)